In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [ ]:
spark = (
    SparkSession
    .builder
    .appName("combustiveis_app")
    .config("extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true,-Divy.cache.dir=/tmp,-Divy.home=/tmp")
    .config("spark.executor.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [ ]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("fs.s3a.fast.upload:", "true")

In [ ]:
BUCKET_LANDING = "<YOUR-LADING-ZONE-BUCKET>"
BUCKET_PROCESSING = "<YOUR-PROCESSING-ZONE-BUCKET>"

In [ ]:
df_landing = (
    spark
    .read
    .format("csv")
    .options(header='true', inferSchema='true', delimiter=';')
    .load(f"s3a://{BUCKET_LANDING}/combustiveis/ca-2016-01.csv")
)

In [ ]:
df_landing.printSchema()

In [ ]:
df_landing.count()

In [ ]:
df_processing = (
    spark
    .read
    .format("parquet")
    .load(f"s3a://{BUCKET_PROCESSING}/combustiveis/")
)

In [ ]:
df_processing.printSchema()

In [ ]:
df_processing.count()

In [ ]:
df_processing_2016_01 =  df_processing.where(f.col('file_name') == f's3a://{BUCKET_LANDING}/combustiveis/ca-2016-01.csv')

In [ ]:
(
    df_processing_2016_01
    .select("estado_sigla", "valor_de_venda", "valor_de_compra")
    .groupBy('estado_sigla')
    .mean().show(30)
)